# Toy Simulacra 1.4

So far, we have built agents that have daily schedules. They can carry out these activities without having to interact with anyone or observe anything about their environment.

## Objectives

In this notebook, we aim to add perception to the agents, allowing them to react to perceived events with volition. Specifically, we want to enable agents to perceive their surroundings, add any events therein to their memory, and react if necessary.

Thus, we need to write functions that do the following:

- perceive: Observes the events and adds them to respective memories.
- retrieve: Retrieves the relevant information from the memory to be used for downstream inferences.
- react / chat: Decides the reaction mode for the agents and how to respond to the focused event.
  

## Setup

Note: Additional print functions have been implemented for clean logging of various aspects of the simulation.

In [1]:
import json
import os
import time
import random
import re
import math
import numpy as np
from pathlib import Path
from operator import itemgetter
from openai import AzureOpenAI
from datetime import datetime, timedelta

from maze import Maze

CLIENT = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

AZURE_MODEL_MAP = {
  'gpt-3.5-turbo-instruct': 'gpt-35-turbo-instruct',
  'gpt-3.5-turbo': 'gpt-35-turbo'
}

GPT_PARAMS = {
    "engine": "gpt-3.5-turbo-instruct", 
    "max_tokens": 500, 
    "temperature": 1.0, 
    "top_p": 1, 
    "stream": False,
    "frequency_penalty": 0, 
    "presence_penalty": 0, 
    "stop": None
}

HOUR_STR = ["00:00 AM", "01:00 AM", "02:00 AM", "03:00 AM", "04:00 AM", 
              "05:00 AM", "06:00 AM", "07:00 AM", "08:00 AM", "09:00 AM", 
              "10:00 AM", "11:00 AM", "12:00 PM", "01:00 PM", "02:00 PM", 
              "03:00 PM", "04:00 PM", "05:00 PM", "06:00 PM", "07:00 PM",
              "08:00 PM", "09:00 PM", "10:00 PM", "11:00 PM"]

TIME_SLEEP_BETWEEN_REQUESTS = 0.1 # seconds

TEMPLATE_FOLDER = Path('./prompt_templates').resolve()
print("template folder", TEMPLATE_FOLDER)

BASE_SIM_FOLDER = Path("./generative_agents/environment/frontend_server/storage/base_the_ville_isabella_maria_klaus/").resolve()
PERSONAS_FOLDER = BASE_SIM_FOLDER / "personas"

PROMPT_LOGFILE = "./prompts_log.txt"
SIM_LOGFILE = "./sim_logs.txt"
CONVERSATION_LOGFILE = "./convo_logs.txt"
FAILSAFE_LOGFILE = "./failsafe_logs.txt"
SCHEDULES_LOGFILE = "./schedules_logfile.txt"
PRINT_SCHEDULE = True
PRINT_PROMPTS = True
PRINT_CONVO = True
PRINT_FAILSAFE = True

CALL_LOGS = {
    "api_calls": 0,
    "fail_safe_counts": {}
}

template folder /Users/gupta/Workspace/tutorials/simulacra/prompt_templates


In [2]:
def print_prompt(fn_name, persona, prompt, response, params, do_not_print=False):
    if not PRINT_PROMPTS:
        return
    
    if do_not_print:
        return
        
    curr_time = persona.scratch.curr_time.strftime('%A %B %d %H:%M:%S')
    with open(PROMPT_LOGFILE, mode="a") as f:
        string = "\n\n" + ">"*50 + "<"*50 + "\n\n" + str(params) + "\n\n"
        print(f"{string}{curr_time}\n{fn_name} --- {persona.name}\n\n --- PROMPT: ---\n{prompt}\n\n--- RESPONSE: ---\n{response}", file=f)

def _normalize(seq):
    _min = min(seq)
    _max = max(seq)
    return [(i - _min) / (1e-6 + _max-_min) for i in seq]

def cos_sim(a,b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def print_to_file(string, logfile):
    with open(logfile, 'a') as f:
        print(string, file=f)

def print_convo(convo, convo_duration_min, convo_summary, curr_time, persona):
    if not PRINT_CONVO:
        return
    string = f"{curr_time.strftime('%A %B %d %H:%M')} -- Initiator: {persona.name}\n\n"
    string += f"Summary: {convo_summary}\n"
    string += f"Time taken (minutes): {str(convo_duration_min)}\n"
    string += "".join([": ".join(i) + "\n" for i in convo])
    with open(CONVERSATION_LOGFILE, 'a') as f:
        print(string, file=f)

def print_failsafe(fn_name, string):
    if not PRINT_FAILSAFE:
        return 

    if fn_name in CALL_LOGS['fail_safe_counts']:
        CALL_LOGS['fail_safe_counts'][fn_name] += 1
    else:
        CALL_LOGS['fail_safe_counts'][fn_name] = 0

    string = f"Fn: {string}"
    with open(FAILSAFE_LOGFILE, 'a') as f:
        print(string, file=f)
    
def print_schedule(string, schedule, curr_time):
    if not PRINT_SCHEDULE:
        return

    string = f"{string}\n"
    t = midnight = datetime(curr_time.year, curr_time.month, curr_time.day)
    for activity, duration in schedule:
        t += timedelta(minutes=duration)
        string += f"{t.strftime('%H:%M')} -- {activity}\n"

    with open(SCHEDULES_LOGFILE, "a") as f:
        print(string, file=f)
        

In [3]:
# Basic prompting helper functions

def get_embedding(text):
    text = text.replace("\n", " ")
    if not text:
        text = "blank"

    response = CLIENT.embeddings.create(
        input = [text],
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

def generate_prompt(prompt_inputs, template_file):
    with open(template_file) as f:
        template = f.read()

    prompt = template.split("<prompt_start>###</prompt_start>")[1]
    for count, input in enumerate(prompt_inputs):
        replace_str = input if input is not None else ""
        prompt = prompt.replace(f"!<INPUT {count}>!", replace_str)

    return prompt.strip()

def prompt_gpt(prompt, parameters):
    try:
        response = CLIENT.completions.create(
            model=AZURE_MODEL_MAP[parameters["engine"]],
            prompt=prompt,
            temperature=parameters["temperature"],
            max_tokens=parameters["max_tokens"],
            top_p=parameters["top_p"],
            frequency_penalty=parameters["frequency_penalty"],
            presence_penalty=parameters["presence_penalty"],
            stream=parameters["stream"],
            stop=parameters["stop"]
        )
        return response.choices[0].text
    except Exception as e:
        print(e)
        return -1


def prompt_gpt4(prompt, parameters):
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant that helps people complete the text either by continuing where they leave or by following the instructions. Don't write unnecessary text. Only the asked tasks.",
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    try:
        response = CLIENT.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=parameters["temperature"],
            max_tokens=parameters["max_tokens"],
            top_p=parameters["top_p"],
            frequency_penalty=parameters["frequency_penalty"],
            presence_penalty=parameters["presence_penalty"],
            stop=parameters["stop"]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(e)
        return -1

def safe_prompting(prompt, parameters, func_clean_up, func_validate=None, repeat=5):
    if func_validate is None:
        def func_validate(response):
            try: func_clean_up(response)
            except: return False
            return True

    for i in range(repeat):
        curr_response = PROMPT_FN(prompt, parameters)
        CALL_LOGS["api_calls"] += 1
        if func_validate(curr_response):
            return func_clean_up(curr_response)
        else:
            time.sleep(TIME_SLEEP_BETWEEN_REQUESTS)

    print(f"{prompt} failed after {repeat} attempt. Returning None.")
    return None


#### DEFINE YOUR PROMPT U=FN
PROMPT_FN = prompt_gpt

### Spatial Memory

In [4]:
class SpatialMemoryTree: 
    def __init__(self, f_saved): 
        self.tree = {}
        self.tree = json.load(open(f_saved))
    
    def print_tree(self): 
        def _print_tree(tree, depth):
            dash = " >" * depth
            if type(tree) == type(list()): 
                if tree:
                  print (dash, tree)
                return 
            
            for key, val in tree.items(): 
                if key: 
                  print (dash, key)
                _print_tree(val, depth+1)
        _print_tree(self.tree, 0)
    
    def get_str_accessible_sectors(self, curr_world):
        return ", ".join(list(self.tree[curr_world].keys()))
    
    def get_str_accessible_sector_arenas(self, sector):
        curr_world, curr_sector = sector.split(":")
        if not curr_sector:
            return ""
        return ", ".join(list(self.tree[curr_world][curr_sector].keys()))

    def get_str_accessible_arena_game_objects(self, arena):
        curr_world, curr_sector, curr_arena = arena.split(":")
        if not curr_arena:
            return ""

        try: 
            x = ", ".join(self.tree[curr_world][curr_sector][curr_arena])
        except:
            x = ", ".join(self.tree[curr_world][curr_sector][curr_arena.lower()])

        return x      

In [5]:
filename = str(PERSONAS_FOLDER / "Isabella Rodriguez/bootstrap_memory/spatial_memory.json")
s_mem = SpatialMemoryTree(filename)
s_mem.print_tree()

 the Ville
 > Hobbs Cafe
 > > cafe
 > > > ['refrigerator', 'cafe customer seating', 'cooking area', 'kitchen sink', 'behind the cafe counter', 'piano']
 > Isabella Rodriguez's apartment
 > > main room
 > > > ['bed', 'desk', 'refrigerator', 'closet', 'shelf']
 > The Rose and Crown Pub
 > > pub
 > > > ['shelf', 'refrigerator', 'bar customer seating', 'behind the bar counter', 'kitchen sink', 'cooking area', 'microphone']
 > Harvey Oak Supply Store
 > > supply store
 > > > ['supply store product shelf', 'behind the supply store counter', 'supply store counter']
 > The Willows Market and Pharmacy
 > > store
 > > > ['behind the pharmacy counter', 'pharmacy store shelf', 'pharmacy store counter', 'grocery store shelf', 'behind the grocery counter', 'grocery store counter']
 > Dorm for Oak Hill College
 > > garden
 > > > ['dorm garden']
 > > common room
 > > > ['common room sofa', 'pool table', 'common room table']
 > Johnson Park
 > > park
 > > > ['park garden']


### Associative memory (Long term memory) 

In [6]:
class ConceptNode:
    node_id: int # bookkeeping
    node_count: int #bookkeeping
    node_type: str # thought / event / chat
    type_count: int # bookkeeping
    depth: int # 

    created: int # 
    expiration: int #

    subject: str # subject usually the agent itself
    predicate: str # 
    object: str # object of this event

    description: str # A full description of the event (usually obtained from LLM)
    embedding_key: str # a key to reference while accessing the embeddings of the event
    embedding: int  # a vector instead. 
    poignancy: int # ??
    keywords: list # keywords to retrieve this node
    filling: int # ??
    
    def __init__(self, **kwargs):
        for key in self.__annotations__:
            setattr(self, key, kwargs.get(key, None)) 
        self.last_accessed = self.created

    def spo_summary(self):
        return (self.subject, self.predicate, self.object)


class AssociativeMemory:
    def __init__(self, folder_name):
        self.id_to_node = dict()

        self.seq_event = []
        self.seq_thought = []
        self.seq_chat = []

        self.kw_to_event = dict()
        self.kw_to_thought = dict()
        self.kw_to_chat = dict()

        x = json.load(open(folder_name + '/kw_strength.json'))
        self.kw_strength_event = x.get('kw_strength_event', dict())
        self.kw_strength_thought = x.get('kw_strength_thought', dict())

        self.embeddings = json.load(open(folder_name + "/embeddings.json"))
        nodes = json.load(open(folder_name + "/nodes.json"))
        for count in range(len(nodes)):
            node_id = f"node_{str(count+1)}"
            node_details = nodes[node_id]

            node_count = node_details['node_count']
            depth = node_details['depth']

            created = datetime.datetime.strptime(node_details['created'], '%Y-%m-%d %H:%M:%S')

            expiration = None
            if node_details['expiration']:
                expiration = datetime.datetime.strptime(node_details['expiration'], '%Y-%m-%d %H:%M:%S')

            subject, predicate, object = node_details['subject'], node_details['predicate'], node_details['object']
            description = node_detals['description']
            embedding_pair = (node_details['embedding_key'], self.embeddings[node_details['embedding_key']])
            poignancy = node_details['poignancy']
            keywords = set(node_details['keywords'])
            filling = node_details['filling']

            self.add_node(node_type, created, expiration, subject, predicate,
                 object, description, keywords, poignancy, embedding_pair, filling)
        
    def add_node(self, node_type, created, expiration, subject, predicate,
                 object, description, keywords, poignancy, embedding_pair, filling):

        node_count = len(self.id_to_node.keys()) + 1
        node_id = f'node_{node_count}'

            
        if node_type == 'chat':
            type_count = len(self.seq_chat) + 1
            depth = 0
        elif node_type == 'event':
            type_count = len(self.seq_event) + 1
            depth = 0  
        elif node_type == 'thought':
            type_count = len(self.seq_thought) + 1
            depth = 1
            if filling:
                depth += max([self.id_to_node[i].depth for i in filling])
                
        node = ConceptNode(node_id=node_id, node_count=node_count, type_count=type_count, node_type=node_type, depth=depth,
                           created=created, expiration=expiration, subject=subject, predicate=predicate,
                           object=object, description=description, embedding_key=embedding_pair[0], poignancy=poignancy,
                           keywords=keywords, filling=filling, embedding=embedding_pair[1])

        if node_type == 'chat':
            self.seq_chat[0:0] = [node]
            cache, cache_strength = self.kw_to_chat, None
        elif node_type == 'event':
            self.seq_event[0:0] = [node]
            cache, cache_strength = self.kw_to_event, self.kw_strength_event
        elif node_type == 'thought':
            self.seq_thought[0:0] = [node]
            cache, cache_strength = self.kw_to_thought, self.kw_strength_thought

        keywords = [i.lower() for i in keywords]
        for kw in keywords:
            if kw in cache:
                cache[kw][0:0] = [node]
            else:
                cache[kw] = [node]

            if cache_strength is not None and f"{predicate} {object}" != "is idle":
                if kw in cache_strength:
                    cache_strength[kw] += 1
                else:
                    cache_strength[kw] = 1

        self.embeddings[embedding_pair[0]] = embedding_pair[1]

        return node
            
        
    def get_summarized_latest_events(self, retention):
        ret_set = set()
        for e_node in self.seq_event[:retention]:
            ret_set.add(e_node.spo_summary())
        return ret_set

    def get_str_seq_events(self):
        ret_str = ""
        for count, event in enumerate(self.seq_event):
            ret_str += f'{"Event", len(self.seq_event) - count, ": ", event.spo_summary(), " -- ", event.description}\n' # returns a string of tuple
        return ret_str    

    def get_str_seq_thoughts(self):
        ret_str = ""
        for count, event in enumerate(self.seq_event):
            ret_str += f'{"Thought", len(self.seq_event) - count, ": ", event.spo_summary(), " -- ", event.description}\n' # returns a string of tuple
        return ret_str  

    def get_str_seq_chats(self):
        ret_str = ""
        for event in self.seq_chat:
            ret_str += f"with {event.object.content} ({event.description})\n"
            ret_str += f"{event.created.strftime('%B %d, %Y, %H:%M:%S')}\n"
            for row in event.filling:
                ret_str += f"{row[0]}: {row[1]}\n"
        return ret_str
            

    def retrieve_relevant_thoughts(self, s, p, o):
        contents = [s, p, o]
        ret = []
        for i in contents:
            if i in self.kw_to_thought:
                ret += self.kw_to_thought[i.lower()]
        return set(ret)

    def retrieve_relevant_events(self, s, p, o):
        contents = [s, p, o]
        ret = []
        for i in contents:
            if i in self.kw_to_event:
                ret += self.kw_to_event[i.lower()]
        return set(ret)
    

### Short term memory (scratch)

In [7]:
class Scratch:
    def __init__(self, filename):
        scratch = json.load(open(filename))

        # PERSONA HYPERPARAMETERS
        self.vision_r = scratch.get('vision_r', 4) # Radius of visible boundaries
        self.att_bandwidth = scratch.get('att_bandwidth', 3) # ??
        self.retention = scratch.get('retention', 5) # ??

        # CORE IDENTITY 
        self.name = scratch.get('name', None)
        self.first_name = scratch.get('first_name', None)
        self.last_name = scratch.get('last_name', None)
        self.age = scratch.get('age', None)
        self.innate = scratch.get('innate', None) # Nature of the person
        self.learned = scratch.get('learned', None) # Learned traits of the person
        self.currently = scratch.get('currently', None) # Any current plans?
        self.lifestyle = scratch.get('lifestyle', None) # Lifestyle of the person
        self.living_area = scratch.get('living_area', None) # General living area; area where they spent the time most outside of work

        # RETRIEVAL RELATED VARIABLES
        self.recency_w = scratch.get('recency_w', 1)
        self.relevance_w = scratch.get('relevance_w', 1)
        self.importance_w = scratch.get('importance_w', 1)
        self.recency_decay = scratch.get('recency_decay', 0.99)

        # WORLD INFORMATION
        self.curr_time = scratch.get('curr_time', datetime.now()) # What's the current time?
        self.curr_tile = scratch.get('curr_tile', None) # Where is the person now?
        self.daily_plan_req = scratch.get('daily_plan_req', '') # What's a typical daily plan?
        tile_filename = '/'.join(filename.split('/')[:-4]) + '/environment/0.json'
        curr_tile = json.load(open(tile_filename))[self.name]
        self.curr_tile = (curr_tile['x'], curr_tile['y'])
        
        # PLANNING VARIABLES
        self.daily_req = scratch.get('daily_req', [])
        self.f_daily_schedule = scratch.get('f_daily_schedule', []) # This is changed every time the hourly activity is decomposed
        self.f_daily_schedule_hourly_org = scratch.get('f_daily_schedule_hourly_org', []) # This remains same as f_daily_schedule

        # ACTIONS OF THE PERSON
        self.act_address = scratch.get('act_address', None)
        self.act_start_time = scratch.get('act_start_time', None)
        self.act_duration = scratch.get('act_duration', None)
        self.act_description = scratch.get('act_description', None)
        self.act_event = scratch.get('act_event', (self.name, None, None)) # See the section on events.

        # CONVERSATION VARIABLES
        self.chatting_with = scratch.get('chatting_with', None)
        self.chat = scratch.get('chat', None)
        self.chatting_with_buffer = scratch.get('chatting_with_buffer', dict())
        self.chatting_end_time = scratch.get('chatting_end_time', None)

    def get_f_daily_schedule_index(self, advance=0, main=True):
        """
        Returns the index of action that is taking place now (advance=0) or sometime in future for a non-zero advance minutes. 
        """ 
        total_time_elapsed = self.curr_time.hour * 60 
        total_time_elapsed += self.curr_time.minute + advance

        ref_list = self.f_daily_schedule if main else self.f_daily_schedule_hourly_org
        elapsed, curr_index = 0, 0
        for task, duration in ref_list:
            elapsed += duration
            if elapsed > total_time_elapsed:
                return curr_index
            curr_index += 1
        return curr_index

    def get_str_iss(self):
        # ISS stands for Identity Stable Set - a bare minimum description of the persona that is used in prompts that need to call on the persona.
        commonset = ""
        commonset += f"Name: {self.name}\n"
        commonset += f"Age: {self.age}\n"
        commonset += f"Innate traits: {self.innate}\n"
        commonset += f"Learned traits: {self.learned}\n"
        commonset += f"Currently: {self.currently}\n"
        commonset += f"Lifestyle: {self.lifestyle}\n"
        commonset += f"Daily plan requirement: {self.daily_plan_req}\n"
        if self.curr_time:
            commonset += f"Current Date: {self.curr_time.strftime('%A %B %d')}\n"
        return commonset

    def add_new_action(self, 
                       action_address,
                       action_duration,
                       action_description,
                       action_event,
                       chatting_with, 
                       chat, 
                       chatting_with_buffer,
                       chatting_end_time,
                       act_obj_description,
                       act_obj_event,
                       act_start_time=None):
        self.act_address = action_address
        self.act_duration = action_duration
        self.act_description = action_description
        self.act_event = action_event

        self.chatting_with = chatting_with
        self.chat = chat
        if chatting_with_buffer:
            self.chatting_with_buffer.update(chatting_with_buffer)

        self.chatting_end_time = chatting_end_time 
        self.act_start_time = self.curr_time # This is the start time of the action
        self.act_path_set = False

    def act_check_finished(self):
        # Returns True if the action has finished
        if not self.act_address:
            return True

        # Compute the end time for the chat
        if self.chatting_with:
            end_time = self.chatting_end_time
        else:
            x = self.act_start_time
            if x.second != 0:
                x = x.replace(second=0)
                x = (x + timedelta(minutes=1))
            end_time = (x + timedelta(minutes=self.act_duration))

        if end_time < self.curr_time:
            return True

        return False

    def act_summarize_str(self):
        start_datetime_str = self.act_start_time.strftime('%A %B %d -- %H:%M %p')
        x = f"[{start_datetime_str}]\n"
        x += f"Activity: {self.name} is {self.act_description}\n"
        x += f"Address: {self.act_address}\n"
        x += f"Duration in minutes (e.g., x min): {str(self.act_duration)} min\n"
        return ret

    def get_curr_event_and_desc(self): 
        if not self.act_address: 
          return (self.name, None, None, None)
        else: 
          return (self.act_event[0], 
                  self.act_event[1], 
                  self.act_event[2],
                  self.act_description)


In [8]:
filename = str(PERSONAS_FOLDER / "Isabella Rodriguez/bootstrap_memory/scratch.json")
stm_mem = Scratch(filename)
print(stm_mem.get_str_iss())

Name: Isabella Rodriguez
Age: 34
Innate traits: friendly, outgoing, hospitable
Learned traits: Isabella Rodriguez is a cafe owner of Hobbs Cafe who loves to make people feel welcome. She is always looking for ways to make the cafe a place where people can come to relax and enjoy themselves.
Currently: Isabella Rodriguez is planning on having a Valentine's Day party at Hobbs Cafe with her customers on February 14th, 2023 at 5pm. She is gathering party material, and is telling everyone to join the party at Hobbs Cafe on February 14th, 2023, from 5pm to 7pm.
Lifestyle: Isabella Rodriguez goes to bed around 11pm, awakes up around 6am.
Daily plan requirement: Isabella Rodriguez opens Hobbs Cafe at 8am everyday, and works at the counter until 8pm, at which point she closes the cafe.



## Integrating Memory into Agents

**Finally, we define a cognitive function in the agent `perceive_and_retrieve_and_focus` and outline functions related to its downstream effects, such as `open_conversation`.**

In [9]:
class Persona:
    def __init__(self, name, folder_mem, curr_time, initiate_plan=True):
        self.name = name
        self.s_mem = SpatialMemoryTree(f"{folder_mem}/bootstrap_memory/spatial_memory.json")
        self.a_mem = AssociativeMemory(f"{folder_mem}/bootstrap_memory/associative_memory")
        self.scratch = Scratch(f"{folder_mem}/bootstrap_memory/scratch.json")
        self.scratch.curr_time = curr_time
        
        if initiate_plan:
            self.generate_day_plan(first_day=True)

    def generate_day_plan(self, first_day):
        # To ensure continuity in plans, we update currently that is akin to reflection on the day and broad goals for the next day
        if not first_day:
            self.scratch.currently = get_new_currently(self)
            
        # Generating persona's daily plan in the short term memory
        self.scratch.daily_req = generate_day_plan(self)
        self.scratch.f_daily_schedule = generate_hourly_schedule(self) # Breaks down the plan into sub units with coherence across the time
        self.scratch.f_daily_schedule_hourly_org[:] = (self.scratch.f_daily_schedule)

        # Adding the broad plan to the long term memory (associative memory)
        curr_date = self.scratch.curr_time.strftime("%A %B %d")
        thought = f"This is {self.scratch.first_name}'s plan for {curr_date}"
        for i in self.scratch.daily_req:
            thought += f" {i},"
        thought = thought[:-1] + "."
        created = self.scratch.curr_time
        expiration =  self.scratch.curr_time + timedelta(days=7) ## EXPIRY = 7 days
        s, p, o = (self.scratch.name, "plan", curr_date)
        keywords = set(["plan"])
        poignancy = 5
        thought_embedding_pair = (thought, get_embedding(thought))
        self.a_mem.add_node("thought", created, expiration, s, p, o,
                               thought, keywords, poignancy, thought_embedding_pair, None)

    def perceive_and_retrieve_and_focus(self, maze):
        if self.scratch.act_description is not None and "sleeping" in self.scratch.act_description:
            return
        perceived = perceive(self, maze) # Adds new information to the memory.
        retrieved = retrieve(self, perceived)
        # Retrieve relevant information from the memory and choose to focus on it, if relevant.
        if not retrieved.keys():
            return
        focus_event = choose_retrieved(persona, retrieved)
        return focus_event

    def advance_one_step(self, maze, personas, curr_time):
        # Obeserve the surroundings, adjust the memory, retrieve relevant information from the memory, chose the event to react to
        focus_event = self.perceive_and_retrieve_and_focus(maze)

        if focus_event:
            react_mode, other = should_react(self, focus_event, personas)
            if react_mode == 2 and self.scratch.act_event[1] != "chat with": 
                # If not chatting already, Open a conversation and adjust the schedule
                self.open_conversation(maze, other)
                
        if self.scratch.curr_time.strftime('%A %B %d') != curr_time.strftime('%A %B %d'):
            self.generate_day_plan(first_day=False)
            
        self.scratch.curr_time = curr_time
        if self.scratch.act_check_finished():
            new_action = determine_action(self, maze) # NOTE: this function call makes changes to f_daily_schedule
            self.scratch.add_new_action(**new_action)

            # determine it's location [don't change anything yet. Changing it here will lead to other personas not observing this event if reuqired]
            action_address = new_action['action_address']
            target_tiles = maze.address_tiles[action_address]
            new_tile = random.sample(list(target_tiles), 1)[0]
            return new_tile
        return None

    def open_conversation(self, maze, other):
        convo, convo_duration_min = generate_convo(maze, self, other)
        convo_summary = generate_convo_summary(self, other, convo)
        print_convo(convo, convo_duration_min, convo_summary, self.scratch.curr_time, self)

        for person, other_person in [(self, other), (other, self)]:
            # new actions for each of self and other
    
            act_address = f"<persona> {other_person.name}"
            act_event = (person.name, "chat with", other_person.name)
            chatting_with = other.name
            chatting_with_buffer = {other.name: 800}
            chatting_end_time = self.scratch.curr_time + timedelta(minutes=convo_duration_min)
            chatting_end_time += timedelta(seconds=60 - chatting_end_time.second)
            
            new_action = {
                "action_address": act_address,
                "action_duration": int(convo_duration_min),
                "action_description": convo_summary,
                "action_event": act_event,
                "chatting_with": chatting_with,
                "chat": convo,
                "chatting_with_buffer": chatting_with_buffer,
                "chatting_end_time":chatting_end_time,
                "act_obj_description": None, 
                "act_obj_event": None
            }
            person.scratch.add_new_action(**new_action)
            # NOTE: In this tutorial, we are not letting conversation affect the new schedules.
            # curr_index = person.scratch.get_f_daily_schedule_index(main=True)
            # new_schedule, start_index, end_index = generate_updated_schedule(person, convo_summary, convo_duration_min)
            # person.scratch.f_daily_schedule[start_index: end_index] = new_schedule

    def get_curr_tile(self):
        return self.scratch.curr_tile

    def move(self, new_tile):
        self.scratch.curr_tile = new_tile
        

### Scheduling 

In [10]:
def generate_day_plan(persona):
    """Produces broad agenda for the day. """
    prompt_template_file = str(TEMPLATE_FOLDER / "day_planning.txt")
    prompt_input = [
        persona.scratch.get_str_iss(),
        persona.scratch.lifestyle,
        persona.scratch.curr_time.strftime("%A %B %d"),
        persona.scratch.first_name
    ]
    
    prompt = generate_prompt(prompt_input, prompt_template_file)
    schedule = safe_prompting(prompt, GPT_PARAMS, lambda x:x)

    print_prompt("generate_daily_plan", persona, prompt, schedule, GPT_PARAMS)

    schedule = prompt + schedule
    schedule = schedule[schedule.find("1)") + 2:]
    schedule = re.split(r'\d+\)', schedule)

    string = f"day high-level schedule -- {persona.name} -- {persona.scratch.curr_time.strftime('%A %B %d %H:%M')}\n"
    print_schedule(string, persona.scratch.f_daily_schedule, persona.scratch.curr_time)

    return [i.strip() for i in schedule]

def generate_hourly_schedule(persona):
    """Uses broad agenda for the day to plan hourly schedule."""
    curr_date = persona.scratch.curr_time.strftime("%A %B %d")
    prompt_template_file = str(TEMPLATE_FOLDER / "hourly_planning.txt")
    # Example of a schedule
    schedule_format = ""
    for hour in HOUR_STR:
        schedule_format += f"[{curr_date} -- {hour}]"
        schedule_format += f" Activity: [Fill in]\n"
    schedule_format = schedule_format[:-1]

    # Broad plan of the persona
    plan_str = f"Here is the orginally intended today's schedule of {persona.scratch.first_name}: "
    for count, activity in enumerate(persona.scratch.daily_req):
        plan_str += f"({str(count+1)}) {activity}, "
    plan_str = plan_str[:-2]
    plan_str += f"\nIf {persona.scratch.first_name} is sleeping, use 'sleeping' as the activity"

    prompt_inputs = [schedule_format, persona.scratch.get_str_iss(), plan_str, None, None]

    # today's prior schedule (needed for coherence)
    activities_list = []
    prior_schedule = "\n"
    for count, hour in enumerate(HOUR_STR):
        # prepare the string for prior schedule
        if count > 0:
            prior_schedule += f"{curr_date} -- {HOUR_STR[count-1]} Acitvity:"
            prior_schedule += f" {persona.scratch.first_name} is {activities_list[count - 1]}\n"
            prompt_inputs[-2] = prior_schedule

        # final prompt to be completed
        final_prompt = f" [{curr_date} -- {hour}] Activity: {persona.scratch.first_name} is"
        prompt_inputs[-1] = final_prompt

        # modify the parameters because we don't need to generate a lot of tokens
        prompt = generate_prompt(prompt_inputs, prompt_template_file)
        params = GPT_PARAMS.copy()
        params['stop'] = ['\n']
        params['temperature'] = 0.5
        params['max_tokens'] = 50
        next_hour_activity = safe_prompting(prompt, params, lambda x:x)

        print_prompt("generate_hourly_schedule", persona, prompt, next_hour_activity, params)
        
        activities_list.append(next_hour_activity.strip())

    # post-processing the output
    compressed_list = [('###', 0)]
    for activity in activities_list:
        if compressed_list[-1][0] == activity:
            compressed_list[-1][1] += 1
        else:
            compressed_list.append([activity, 1])
    compressed_list.pop(0)

    string = f"hourly schedule -- {persona.name} -- {persona.scratch.curr_time.strftime('%A %B %d %H:%M')}\n"
    print_schedule(string, persona.scratch.f_daily_schedule, persona.scratch.curr_time)
    
    return [(x, y*60) for x,y in compressed_list]

def generate_task_decompose(persona):
    """Generates 5 min increments of the current action for duration of that action."""
    curr_date = persona.scratch.curr_time.strftime("%A %B %d")
    prompt_template_file = str(TEMPLATE_FOLDER / "decompose_task.txt")
    
    curr_f_org_index = persona.scratch.get_f_daily_schedule_index(main=False) # gets from f_daily_schedule_hourly_org
    # print(curr_f_org_index, persona.scratch.f_daily_schedule_hourly_org, len(persona.scratch.f_daily_schedule_hourly_org))
    # Prepare a summary string to capture an hour before and an hour after the current action event
    summary_str = f"Today is {curr_date}. From "
    for index in [curr_f_org_index- 1, curr_f_org_index, curr_f_org_index+1]:
        if index >= len(persona.scratch.f_daily_schedule_hourly_org) or index < 0:
            continue

        start_min = sum(i[1] for i in persona.scratch.f_daily_schedule_hourly_org[:index])
        action, time_elapsed = persona.scratch.f_daily_schedule_hourly_org[index]
        start_time = datetime.strptime("00:00:00", "%H:%M:%S") + timedelta(minutes=start_min)
        end_time = start_time + timedelta(minutes=time_elapsed)

        start_time_str, end_time_str = start_time.strftime('%H:%M%p'), end_time.strftime('%H:%M%p')
        summary_str += f"{start_time_str} ~ {end_time_str}, {persona.name} is planning {action}, "

        if index == curr_f_org_index: # We are interested in decomposing the activity at curr_f_org_index
            curr_time_range, curr_time_duration, curr_action_desc = f"{start_time_str} ~ {end_time_str}", str(time_elapsed), action
            total_time_range = time_elapsed

    summary_str = summary_str[:-2] + ". "

    prompt_inputs = [
        persona.scratch.get_str_iss(),
        summary_str,
        persona.scratch.first_name,
        curr_action_desc,
        curr_time_range,
        curr_time_duration
    ]

    params = GPT_PARAMS.copy()
    params['temperature'] = 0.8 ## Empirically, so that it doesn't deviate from the output format.
    prompt  = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, params, lambda x:x)
    
    print_prompt("generate_task_decompose", persona, prompt, response, params)

    full_str = prompt + response
    rem_str = full_str.split("---")[3]
    schedule = re.split(r'\d+\>', rem_str)
    schedule = [i.strip() for i in schedule if i.strip()]

    # post-process this schedule to 5 min increments
    activities = [["dummy", -1]]
    for activity in schedule:
        try:
            task, rest = activity.split("(duration in minutes:")
        except:
            task, rest = activity.split("(duration in minutes") # Failure prevention
            
        if "," not in rest: # FAIL PREVENTION: Sometimes prompt might not give ", minutes left: xx)" in the end as prompted.
            duration = int(rest[:-1])
        else:
            duration = int(rest.split(",")[0])

        activities.append([task.strip(), duration])
    activities = activities[1:]

    # Making sure that the activities fall in the time range.    
    lagging_sum, duration_sum, idx = 0, 0, 0
    output = []
    for task, duration in activities:
        duration_sum += duration
        if duration_sum <= total_time_range:
            output.append([task, duration])
        else:
            output.append([task, total_time_range - lagging_sum])
        idx += 1
        lagging_sum += duration

    return output

def determine_action(persona, maze):
    """Updates the agent's activity as well as decompose if necessary."""
    def determine_decompose(act_desc, act_dura):
        if "sleeping" in act_desc:
            return False
        if act_dura < 60:
            return False 
        return True
    
    curr_action_index = persona.scratch.get_f_daily_schedule_index()

    act_desc, act_dura =  persona.scratch.f_daily_schedule[curr_action_index]
    if determine_decompose(act_desc, act_dura):
        persona.scratch.f_daily_schedule[curr_action_index: curr_action_index+1] = (
            generate_task_decompose(persona) # GPT
        )
        string = f"decomposed -- {persona.name} -- {persona.scratch.curr_time.strftime('%A %B %d %H:%M')}\n"
        string += f"Current action: {act_desc}\nDuration: {act_dura}"
        print_schedule(string, persona.scratch.f_daily_schedule, persona.scratch.curr_time)
        
        # to add up minutes
        total_time_accounted = sum(i[1] for i in persona.scratch.f_daily_schedule)
        if total_time_accounted < 1440:
            persona.scratch.f_daily_schedule += [["sleeping", 1440 - total_time_accounted]]

    act_desc, act_dura = persona.scratch.f_daily_schedule[curr_action_index]

    # Now we determine this action's location to execute
    act_world = maze.access_tile(persona.scratch.curr_tile)['world']
    act_sector = generate_action_sector(act_desc, persona, maze, 
                                         curr_determined_address=act_world) # GPT
    act_arena = generate_action_sector_arena(act_desc, persona, maze, 
                                        curr_determined_address=f"{act_world}:{act_sector}") # GPT
    act_object = generate_action_sector_arena_object(act_desc, persona, maze, 
                                                     curr_determined_address=f"{act_world}:{act_sector}:{act_arena}") # GPT
    new_address = f"{act_world}:{act_sector}:{act_arena}" 
    new_address += f":{act_object}" if act_object else ""
    act_event = generate_action_event_triple(act_desc, persona)
    act_obj_desc = None
    act_obj_event = None

    return {
        "action_address": new_address,
        "action_duration": int(act_dura),
        "action_description": act_desc,
        "action_event": act_event,
        "chatting_with": None, "chat": None, "chatting_with_buffer": None, "chatting_end_time": None,
        "act_obj_description": act_obj_desc,
        "act_obj_event": act_obj_event,
    }
                                   
def generate_action_sector(action, persona, maze, curr_determined_address=None):
    """Returns the appropriate sector to carry out that action."""
    prompt_template_file = str(TEMPLATE_FOLDER / "determine_action_sector.txt")
    curr_tile = persona.scratch.curr_tile
    tile_info = maze.access_tile(curr_tile)
    world, curr_sector = tile_info['world'], tile_info['sector']
    all_sectors = persona.s_mem.get_str_accessible_sectors(f"{world}")
    curr_sector_arenas = persona.s_mem.get_str_accessible_sector_arenas(f"{world}:{curr_sector}")
    
    living_area = persona.scratch.living_area
    living_area_sector = living_area.split(":")[1]
    living_area_arenas = persona.s_mem.get_str_accessible_sector_arenas(f"{world}:{living_area_sector}")
    prompt_inputs = [
        persona.scratch.name,
        living_area_sector,
        living_area_arenas,
        curr_sector,
        curr_sector_arenas,
        all_sectors,
        action
    ]
    
    params = GPT_PARAMS.copy()
    params['max_tokens'] = 20
    params['temperature'] = 0
    params['top_p'] = 1
    prompt  = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, params, lambda x:x)

    print_prompt("generate_action_sector", persona, prompt, response, params)

    return response.split("}")[0]

def generate_action_sector_arena(action, persona, maze, curr_determined_address):
    """Returns the appropriate arena within the sector to carry out that action."""
    prompt_template_file = str(TEMPLATE_FOLDER / "determine_action_arena.txt")
    new_sector = curr_determined_address.split(":")[1]
    new_possible_arenas = persona.s_mem.get_str_accessible_sector_arenas(curr_determined_address)
    prompt_inputs = [
        persona.scratch.name,
        new_sector,
        new_possible_arenas,
        action     
    ]
    params = GPT_PARAMS.copy()
    params['max_tokens'] = 20
    params['temperature'] = 0
    params['top_p'] = 1
    prompt  = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, params, lambda x:x)
    
    print_prompt("generate_action_sector", persona, prompt, response, params)

    if response.split("}")[0] not in [x.strip() for x in new_possible_arenas.split(",")]:
        object = random.sample(new_possible_arenas.split(","), 1)[0].strip()
        
        string = ">"*50 + "<"*50 + "\n" + f"new_possible_arenas @ {persona.name} @ {curr_determined_address} @ {action} --> {object}\n"
        string += f"response: {response}\n"
        string += f"failed: response not in {new_possible_arenas.split(',')}\n"
        string += "default: random.sample \n\n"
        print_failsafe("generate_action_sector_arena", string)
    
    return response.split("}")[0]

def generate_action_sector_arena_object(action, persona, maze, curr_determined_address):
    """Returns the appropriate object within the arena to carry out that action."""
    prompt_template_file = str(TEMPLATE_FOLDER / "determine_action_object.txt")
    possible_objects = persona.s_mem.get_str_accessible_arena_game_objects(curr_determined_address)
    prompt_inputs = [
        f"{action}",
        possible_objects    
    ]
    params = GPT_PARAMS.copy()
    params['max_tokens'] = 20
    params['temperature'] = 0.1
    params['top_p'] = 1
    params['stop'] = ["\n"]
    prompt  = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, params, lambda x:x)

    print_prompt("generate_action_sector_arena_object", persona, prompt, response, params)
    
    # Fail safe mechanism
    if response.strip() not in [x.strip() for x in possible_objects.split(",")]:
        object = random.sample(possible_objects.split(","), 1)[0].strip()
        
        string = ">"*50 + "<"*50 + "\n" + f"generate_action_sector_arena_object @ {persona.name} @ {curr_determined_address} @ {action} --> {object}\n"
        string += f"response: {response}\n"
        string += f"failed: response not in {possible_objects.split(',')}\n"
        string += "default: random.sample \n\n"
        print_failsafe("generate_action_sector_arena_object", string)
        
        return object

    return response.strip()

def generate_action_event_triple(action, persona):
    """Returns the (subject, predicate, object) decomposition corresponding to the action."""
    prompt_template_file = str(TEMPLATE_FOLDER / "generate_event_triplet.txt")
    prompt_inputs = [
        persona.scratch.name,
        action.lower()
    ]  
    params = GPT_PARAMS.copy()
    params['max_tokens'] = 50
    params['temperature'] = 0
    params['top_p'] = 1
    prompt  = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, params, lambda x:x)

    print_prompt("generate_action_event_triple", persona, prompt, response, params)

    full_str = prompt + response
    output = full_str.split("---")[-1].split("Output:")[-1].strip()[1:]

    output = [i.strip() for i in output.split(")")[0].split(",")]
    if len(output) != 3:
        output = [persona.scratch.name, 'is', output[-1]]

        string = ">"*50 + "<"*50 + "\n" + f"generate_action_event_triple @ {persona.name} @ {action} --> {output}\n"
        string += f"response: {response}\n"
        string += f"failed: len(output) != 3\n"
        string += "default: output = [persona.scratch.name, 'is', output[-1]] \n\n"
        print_failsafe("generate_action_event_triple", string)
    
    return output

def get_new_currently(persona):
    """Reflects on the day's activity and returns a new `currently` for persona to take on. """
    name = persona.scratch.name 
    curr_day = persona.scratch.curr_time.strftime("%A %B %d")
    queries = [
        f"{name}'s plan for {curr_day}",
        f"Important recent events for {name}'s life."
    ]
    retrieved = extract_relevant_nodes(persona, queries, count=30)

    # Add statements about the retrieved nodes
    statements = "[Statements]\n"
    for query, nodes in retrieved.items():
        for node in nodes:
            statements += f"{node.created.strftime('%A %B %d -- %H:%M %p')}: {node.embedding_key}\n"

    # Create a broad agenda for the next day
    planning_prompt = f"""{statements}
    Given the statements above, is there anything that {name} should remember as they plan for *{curr_day}*?
    If there is any scheduling information, be as specific as possible (including date, time, and location if stated in the statement).\n
    Write the response from {name}'s perspective.
    """
    params = GPT_PARAMS.copy()
    params['model'] = "gpt-3.5-turbo"
    params['max_tokens'] = 1000
    params['temperature'] = 0.8
    plan_note = safe_prompting(planning_prompt, params, lambda x:x)

    print_prompt("get_new_currently --> plan_note", persona, planning_prompt, plan_note, params)

    thought_prompt = f"""{statements}
    Given the statements above, how might we summarize {name}'s feelings about their days up to now?\n
    Write the response from {name}'s perspective.
    """
    thought_note = safe_prompting(thought_prompt, params, lambda x:x)

    print_prompt("get_new_currently --> thought_note", persona, thought_prompt, thought_note, params)

    prev_currently = persona.scratch.currently
    prev_day = persona.scratch.curr_time - timedelta(days=1)
    prev_day = prev_day.strftime('%A %B %d')
    update_currently_prompt = f"""
    {name}'s status from {prev_day}: {prev_currently}\n\n
    {name}'s thoughts at the end of {prev_day}: {plan_note} {thought_note}\n\n
    It is now {curr_day}. Given the above, write {name}'s status for {curr_day} that reflects {name}'s thoughts at the end of {curr_day}.
    Write this in third-person talking about {name}.
    If there is any scheduling information, be as specific as possible (include date, time, and location if stated in the statement).\n\n
    Follow this format below:\nStatus: <new_status>
    """
    new_currently = safe_prompting(update_currently_prompt, params, lambda x:x)

    print_prompt("get_new_currently --> new_currently", persona, update_currently_prompt, new_currently, params)

    return new_currently

def extract_relevant_nodes(persona, queries, count=30):
    """Retrieves nodes from agent's memory relevant to all `query` in `queries`."""
    nodes = []
    for node in persona.a_mem.seq_thought + persona.a_mem.seq_event + persona.a_mem.seq_chat:
        if "idle" not in node.embedding_key:
            nodes.append([node.last_accessed, node])
    nodes = sorted(nodes, key=lambda x:x[0])
    nodes = [node for _, node in nodes]

    persona_receny_w = persona.scratch.recency_decay
    recency = _normalize([persona_receny_w**i for i in range(1, len(nodes) + 1)])
    importance = _normalize([node.poignancy for node in nodes])
    
    retrieved = dict()
    v1, v2, v3 = persona.scratch.recency_w, persona.scratch.relevance_w, persona.scratch.importance_w
    w1, w2, w3 = 0.5, 3, 2 ## HARD CODED WEIGHTS 
    for query in queries:
        query_embedding = get_embedding(query)
        node_relevance = _normalize([cos_sim(node.embedding, query_embedding) for node in nodes])
        node_relevance = [x*v1*w1 + y*v2*w2 + z*v3*w3 for x,y,z in zip(recency, importance, node_relevance)]
        top_nodes = sorted([(val, idx) for idx, val in enumerate(node_relevance)], key=lambda x:x[0])[-count:]
        for _, idx in top_nodes:
            nodes[idx].last_accessed = persona.scratch.curr_time
        retrieved[query] = [nodes[idx] for _, idx in top_nodes]

    return retrieved


## Cognitive Functions
Enhancing agency in simulation agents.

In this section, we outline the cognitive functions of an agent, focusing on how these functions record the surrounding environment as perceptions, document interactions (chats) with other agents, add them to appropriate memory structures, and retrieve relevant memories as needed.

### Perceive
Perception in agents is influenced by several design factors. For example, agents only observe tiles within their perception radius (`vision_r`). Moreover, among all perceived events, an agent can focus on only a limited number (`att_bandwidth`) of them. Once determined, these events are added to the associative memory, categorized either as chats or events.

It's important to note that each event is assigned a score for its importance (`poignancy`). These scores are generated by LLMs using specific prompts, which are then utilized in the retrieval function defined later.

In [11]:
def perceive(persona, maze):
    curr_tile = persona.scratch.curr_tile
    curr_arena_address = maze.get_tile_path(curr_tile, level='arena')
    nearby_tiles = maze.get_nearby_tiles(persona.scratch.curr_tile, persona.scratch.vision_r)

    percept_events_set = set()
    percept_events_list = []

    # We add new objects to the spatial memory of the agent
    # We also take notice of new events and their distance from the agent
    for i in nearby_tiles:
        tile_info = maze.access_tile(i)

        world, sector = tile_info.get('world', None), tile_info.get('sector', None)
        arena, object = tile_info.get('arena', None), tile_info.get('object', None)
        
        # Initialize spatial memory of persona with this tile
        if world and world not in persona.s_mem.tree: 
            persona.s_mem.tree[world] = {}

        if sector and sector not in persona.s_mem.tree[world]: 
            persona.s_mem.tree[world][sector] = {}

        if arena and arena not in persona.s_mem.tree[world][sector]: 
            persona.s_mem.tree[world][sector][arena] = []

        if object and object not in persona.s_mem.tree[world][sector][arena]: 
            persona.s_mem.tree[world][sector][arena].append(object)

        tile_arena_address = maze.get_tile_path(i, level="arena")
        if tile_info['events'] and tile_arena_address == curr_arena_address:
            dist = math.dist(i, curr_tile)

            # add events
            for event in tile_info['events']:
                if event not in percept_events_set:
                    percept_events_list += [[dist, event]]
                    percept_events_set.add(event)

    # Agent only retains events based on their distance
    percept_events_list = sorted(percept_events_list, key=itemgetter(0))
    perceived_events = [event for dist, event in percept_events_list[:persona.scratch.att_bandwidth]]

    # Add these events (if new) to appropriate memory structure of the agent (associative memory and scratch)
    latest_events = persona.a_mem.get_summarized_latest_events(persona.scratch.retention)
    ret_events = []
    # print(f"LATEST EVENTS: {latest_events}\n\nPERCEIVED EVENTS:{perceived_events}")
    for p_event in perceived_events:
        subject, predicate, object, desc = p_event
        if not predicate:
            predicate, object, desc = "is", "idle", "idle"
        desc = f"{subject.split(':')[-1]} is {desc}"
        p_event = (subject, predicate, object)
        
        if p_event not in latest_events:
            
            sub = p_event[0] if ":" not in p_event[0] else p_event[0].split(":")[-1]
            obj = p_event[2] if ":" not in p_event[2] else p_event[2].split(":")[-1]
            keywords = set([sub, obj])

            # embeddings to represent these events (think of neural encodings in brain, maybe)
            desc_embedding_in = desc
            if "(" in desc:
                # "(xyz)" --> ("xyz")
                desc_embedding_in = (desc_embedding_in.split("(")[1].split(")")[0].strip())

            if desc_embedding_in in persona.a_mem.embeddings:
                event_embedding = persona.a_mem.embeddings[desc_embedding_in]
            else:
                event_embedding = get_embedding(desc_embedding_in)
            event_embedding_pair = (desc_embedding_in, event_embedding)

            # poignancy
            event_poignancy = generate_poignancy_score(persona, "event", desc_embedding_in)

            # Add chats to the memory
            created, expiration = persona.scratch.curr_time, None
            chat_node_ids = []
            if p_event[0] == persona.name and p_event[1] == "chat with":
                curr_event = persona.scratch.act_event
                act_desc = persona.scratch.act_description
                if act_desc in persona.a_mem.embeddings:
                    chat_embedding = persona.a_mem.embeddings[act_desc]
                else:
                    chat_embedding = get_embedding(act_desc)
                chat_embedding_pair = (act_desc, chat_embedding)
                chat_poignancy = generate_poignancy_score(persona, "chat", act_desc)
                chat_node = persona.a_mem.add_node("chat", created, expiration, 
                                                   curr_event[0], curr_event[1], curr_event[2],
                                                   act_desc, keywords, chat_poignancy, chat_embedding_pair,
                                                   persona.scratch.chat)
                chat_node_ids = [chat_node.node_id]            
            
            new_node_in_mem = persona.a_mem.add_node('event', created, expiration, subject, predicate,
                 object, desc, keywords, event_poignancy, event_embedding_pair, chat_node_ids) 
            ret_events.append(new_node_in_mem)

    return ret_events 

def generate_poignancy_score(persona, event_type, description):
    """Returns the importance or poignancy of the event based on its description and the agent characteristics. """
    if "is idle" in description:
        return 1

    prompt_template_file = str(TEMPLATE_FOLDER / f"generate_{event_type}_poignancy_score.txt")
        
    prompt_inputs = [
        persona.name,
        persona.scratch.get_str_iss(),
        description,
    ]

    prompt = generate_prompt(prompt_inputs, prompt_template_file)
    score = safe_prompting(prompt, GPT_PARAMS, lambda x:x)
    
    print_prompt(f"generate_poignancy_score -- {event_type}", persona, prompt, score, GPT_PARAMS)

    try: return int(score)
    except:
        string = ">"*50 + "<"*50 + "\n" + f"generate_poignancy_score -- {event_type} @ {persona.name} @ {description} --- Response: {score}\n"
        string += f"response: {score}\n"
        string += f"failed: int(score)\n"
        string += "default: 0\n\n"
        print_failsafe("generate_poignancy_score", string)

        return 0

### Retrieve

Each perceived event is a `ConceptNode` in the agent's memory. These evetns invoke certain thoughts and events in the generative agent. 
The following function does the job of retrieving (`retrieve`) and determining what's worth focusing on (`choose_retrieved`). 

In [12]:
def retrieve(persona, perceived):
    """Returns relevant events and thoughts in the agent memory corresponding to each event in `perceived`."""
    retrieved = dict()
    for node in perceived: 
        retrieved[node.description] = dict()
        retrieved[node.description]["curr_event"] = node
        
        relevant_events = persona.a_mem.retrieve_relevant_events(
                            node.subject, node.predicate, node.object)
        retrieved[node.description]["events"] = list(relevant_events)
        
        relevant_thoughts = persona.a_mem.retrieve_relevant_thoughts(
                              node.subject, node.predicate, node.object)
        retrieved[node.description]["thoughts"] = list(relevant_thoughts)
    
    return retrieved

def choose_retrieved(persona, retrieved):
    """Chooses one of the retrieved events. Currently, it only retrieves if there are other agents around. """
    relevant = []
    # conditions are as used in the simulacra code. See _choose_retrieved in plan.py
    for event_desc, info in retrieved.items():
        node = info['curr_event']
        if node.subject == persona.name: 
            continue
        if (":" not in node.subject and "is idle" not in event_desc):
            relevant.append(info)

    if relevant:
        # print(f"RELEVANT: {relevant}")
        return random.choice(relevant)
    return None

### React / Chat

This section outlines the decision-making process regarding an agent's response to a focused event (`should_react`). The determination of whether and how an agent should react (`generate_reaction_type`) encompasses several outcomes: taking no action, continuing its current activity, or initiating a conversation with another agent.

Should the agent opt to engage in a conversation, we have functions designed to facilitate this interaction between two agents (`generate_convo`). Following the completion of the conversation, a summarized version (`generate_convo_summary`) is incorporated into the agent’s short-term memory. Subsequently, the perception function archives this summary into the long-term memory along with the conversation's details.

Note: Ideally, any such conversation would be reflected in the agent's schedule. However, for simplicity, we will not account for changes to the schedule resulting from these interactions.

In [13]:
def should_react(persona, focused_event, all_personas):
    """Determines iff there should be a reaction. If so, what type of reaction? 0=do nothing, 1=continue with their work (~0), 2=chat."""
    
    if "<waiting>" in persona.scratch.act_address:
        return 0, None
    
    event_node = focused_event['curr_event']

    # In this notebook, we don't converse with other agents --- see the next notebook for that.
    if ":" in event_node.subject: # this is an object --- we don't consider any reaction to the objects in this simulation
        return 0, None

    other_persona = [p for p in all_personas if p.name == event_node.subject]
    if not other_persona:
        return 0, None

    react_mode = generate_reaction_type(persona, focused_event, other_persona[0])
    return react_mode, other_persona[0]

def generate_reaction_type(persona, focused_event, other_persona):
    """Prompts LLMs to determine the reaction type in response to `focus_event`. """
    prompt_template_file = str(TEMPLATE_FOLDER / "generate_reaction_type.txt")
    curr_time_str = persona.scratch.curr_time.strftime("%B %d, %Y, %H:%M:%S %p")
    focused_event_description = focused_event['curr_event'].description

    name = persona.name
    # relevant events from persona's memory
    context_str = f"{name} just observed {focused_event_description}."
    context_str += f"These are the past relevant events in {name}'s experience:\n"
    for node in focused_event['events']:
        context_str += f"{node.description}. "
    
    # relevant thoughts from persona's memory
    context_str += f"\nThese are the relevant thoughts in {name}'s mind:"
    for node in focused_event['thoughts']:
        context_str += f"{node.description}. "

    # what is persona doing right now
    persona_action_desc = persona.scratch.act_description

    # what is other_persona doing right now
    other_persona_action_desc = other_persona.scratch.act_description
    
    prompt_inputs = [
        context_str,
        curr_time_str,
        persona_action_desc,
        other_persona_action_desc,
        name,
        other_persona.scratch.name,
        persona.scratch.get_str_iss(),
        other_persona.scratch.get_str_iss(),
    ]

    prompt = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, GPT_PARAMS, lambda x:x)

    print_prompt(f"generate_reaction_type", persona, prompt, response, GPT_PARAMS)
    
    try:
        return int(response.split(":")[0].split("Option ")[-1])
    except:
        string = ">"*50 + "<"*50 + "\n" + f"generate_reaction_type -- {persona.scratch.name} -- {other_persona.scratch.name} -- {curr_time_str}\n"
        string += f"response: {response}\n"
        string += "failed: int(response.split(':')[0].split('Option ')[-1])\n"
        string += "default: Option 2\n\n"
        print_failsafe("generate_reaction_type", string)
        return 2

def generate_convo(maze, persona, other_persona):
    """Generates conversation."""
    convo = simulate_convo(maze, persona, other_persona)

    all_utt = ""
    for row in convo:
        speaker = row[0]
        utt = row[1]
        all_utt += f"{speaker}: {utt}\n"

    # Heuristic: 30 words per minute where each word has 8 characters on average
    # Note: usual statistics is different; 120 words per minute in normal conversation. 
    convo_duration_min = math.ceil(int(len(all_utt)/8 / 30))
    return convo, convo_duration_min


def simulate_convo(maze, persona, other_persona):
    """Simulates conversation between two agents."""
    curr_chat = []
    for i in range(8):
        for speaker, listener in [(persona, other_persona), (other_persona, persona)]:
            focal_points = [f"{listener.scratch.name}"]
            retrieved_nodes = extract_relevant_nodes(speaker, focal_points, count=50) # What does agent know about the other persona?
            relationship = generate_summarize_relationship(speaker, listener, retrieved_nodes) # summarize relationship between them
    
            # Create new focal points for the new conversation
            focal_points = [
                f"{relationship}",
                f"{listener.scratch.name} is {listener.scratch.act_description}"
            ]
            last_chat = [": ".join(i) + "\n" for i in curr_chat[-4:]]
            if last_chat:
                focal_points.append("".join(last_chat))
            retrieved_nodes = extract_relevant_nodes(speaker, focal_points, count=15)
            utterance, end = generate_one_utterance(maze, speaker, listener, retrieved_nodes, curr_chat)
            curr_chat += [[speaker.scratch.name, utterance.strip()]]

            if end:
                break
    
        if end: 
            break
    
    return curr_chat

def generate_summarize_relationship(persona, other_persona, retrieved_nodes):
    """Summarizes relationship between the to agents."""
    prompt_template_file = str(TEMPLATE_FOLDER / "summarize_relationship.txt")
    all_embedding_keys = list()
    all_embedding_keys = [f"{i.embedding_key}\n" for key, val in retrieved_nodes.items() for i in val]
    all_embedding_keys_str = "".join(all_embedding_keys)
    prompt_inputs = [
        all_embedding_keys_str,
        persona.scratch.name,
        other_persona.scratch.name
    ]

    prompt = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, GPT_PARAMS, lambda x:x)

    print_prompt(f"generate_summarize_relationship", persona, prompt, response, GPT_PARAMS)
    return response

def generate_one_utterance(maze, speaker, listener, retrieved_nodes, curr_chat):
    """Generates the utterance of the `speaker` in response to `listener` and the current context. """
    prompt_template_file = str(TEMPLATE_FOLDER / "generate_one_utterance.txt")
    curr_context = f"""
    {speaker.scratch.name} was {speaker.scratch.act_description} when {speaker.scratch.name} saw {listener.scratch.name}
    in the middle of {listener.scratch.act_description}.\n{speaker.scratch.name} is initiating a conversation with {listener.scratch.name}.
    """
    retrieved_memory_str = [f"- {v.description}\n" for key, vals in retrieved_nodes.items() for v in vals]
    # Adding the last conversation between the two
    prev_convo = ""
    for i in speaker.a_mem.seq_chat:
        if i.object == listener.scratch.name:
            mins_ago = int((speaker.scratch.curr_time - i.created).total_seconds()/60)
            prev_convo = f"{str(mins_ago)} minutes ago, {speaker.scratch.name} and {listener.scratch.name} were already {i.description}. This context takes place after that conversation."
            break

    tile_info = maze.access_tile(speaker.scratch.curr_tile)
    curr_location = f"{tile_info['arena']} in {tile_info['sector']}"
    if len(curr_chat) == 0:
        convo_str = "[The conversation has not started yet -- start it!]"
    else:
        convo_str = [": ".join(i) + "\n" for i in curr_chat]
        
    prompt_inputs = [
        speaker.scratch.get_str_iss(),
        speaker.scratch.name,
        "".join(retrieved_memory_str),
        prev_convo,
        curr_location,
        curr_context,
        listener.scratch.name,
        "".join(convo_str),
    ]
    prompt = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, GPT_PARAMS, lambda x:x)
    response = response.strip()
    print_prompt(f"generate_one_utterance -- speaker: {speaker.scratch.name} -- listener: {listener.name}", speaker, prompt, response, GPT_PARAMS)

    try:
        x = response.strip().split("\n")
        utt = x[0].split(f"{speaker.scratch.name}:")[-1].strip()
        if len(x) == 2 and x[1] != "" :
            end = False if "False" in x[1] else True
        else:
            end = True
    except Exception as e:
        string = ">"*50 + "<"*50 + "\n" + f"generate_one_utterance -- speaker: {speaker.scratch.name} -- listener: {listener.name}. returning dict()\n"
        string += f"response: {response}\n"
        string += f"failed: response.split(f'{speaker.scratch.name}:')[-1] -- {e}\n"
        string += "default: utt:'' end:True \n\n"
        print_failsafe("generate_one_utterance", string)
        utt, end = "", True

    return utt, end

def generate_convo_summary(persona, other_persona, convo):
    """Summarizes the conversation between two agents."""
    prompt_template_file = str(TEMPLATE_FOLDER / "summarize_convo.txt")
    convo_str  = [": ".join(row) + "\n" for row in convo]
    prompt_inputs = [
        "".join(convo_str),
        persona.scratch.name,
        other_persona.scratch.name,
    ]

    prompt = generate_prompt(prompt_inputs, prompt_template_file)
    response = safe_prompting(prompt, GPT_PARAMS, lambda x:x)

    print_prompt(f"generate_convo_summary -- intiator: {persona.scratch.name} -- other: {other_persona.name}", persona, prompt, response, GPT_PARAMS)

    return response

## Simulation loop

In [14]:
# Clear file contents
open(SIM_LOGFILE, 'w').close()
open(PROMPT_LOGFILE, 'w').close() 
open(FAILSAFE_LOGFILE, 'w').close()
open(CONVERSATION_LOGFILE, 'w').close() 
open(SCHEDULES_LOGFILE, 'w').close() 
CALL_LOGS = {'api_calls': 0, 'fail_safe_counts': {}}

maze = Maze("the Ville")
curr_time = sim_start_time = datetime(2024, 2, 13, 0, 0, 0) # Start at midnight
seconds_per_step = 10 * 60 # 10 minutes
n_steps = 180

personas = []
for persona_folder in PERSONAS_FOLDER.iterdir():
    personas.append(Persona(persona_folder.name, persona_folder, curr_time, initiate_plan=True))

step = 0
personas = personas[:3]
movements = {}
while step < n_steps:
    
    # update and execute activities
    for persona in personas:
        curr_tile = persona.get_curr_tile()
        new_tile = persona.advance_one_step(maze, personas, curr_time)
        movements[persona.name] = new_tile

    # update location
    for persona in personas:
        new_tile = movements[persona.name]
        if new_tile:            
            maze.remove_subject_events_from_tile(persona.name, curr_tile)
            maze.add_event_from_tile(persona.scratch.get_curr_event_and_desc(), new_tile)
            persona.move(new_tile)
        
        tile_path = maze.get_tile_path(persona.scratch.curr_tile, level='object')
        string = f"{curr_time.strftime('%H:%M')} {persona.name} {persona.scratch.curr_tile} {persona.scratch.act_description} {persona.scratch.act_address} {tile_path}"
        print_to_file(string, SIM_LOGFILE)
    
    step += 1
    curr_time = sim_start_time + timedelta(seconds=seconds_per_step*step)
    print(curr_time.strftime("%H:%M"),)

00:10
00:20
00:30
00:40
00:50
01:00
01:10
01:20
01:30
01:40
01:50
02:00
02:10
02:20
02:30
02:40
02:50
03:00
03:10
03:20
03:30
03:40
03:50
04:00
04:10
04:20
04:30
04:40
04:50
05:00
05:10
05:20
05:30
05:40
05:50
06:00
06:10
06:20
06:30
06:40
06:50
07:00
07:10
07:20
07:30
07:40
07:50
08:00
08:10
08:20
08:30
08:40
08:50
09:00
09:10
09:20
09:30
09:40
09:50
10:00
10:10
10:20
10:30
10:40
10:50
11:00
11:10
11:20
11:30
11:40
11:50
12:00
12:10
12:20
12:30
12:40
12:50
13:00
13:10
13:20
13:30
13:40
13:50
14:00
14:10
14:20
14:30
14:40
14:50
15:00
15:10
15:20
15:30
15:40
15:50
16:00
16:10
16:20
16:30
16:40
16:50
17:00
17:10
17:20
17:30
17:40
17:50
18:00
18:10
18:20
18:30
18:40
18:50
19:00
19:10
19:20
19:30
19:40
19:50
20:00
20:10
20:20
20:30
20:40
20:50
21:00
21:10
21:20
21:30
21:40
21:50
22:00
22:10
22:20
22:30
22:40
22:50
23:00
23:10
23:20
23:30
23:40
23:50
00:00
00:10
00:20
00:30
00:40
00:50
01:00
01:10
01:20
01:30
01:40
01:50
02:00
02:10
02:20
02:30
02:40
02:50
03:00
03:10
03:20
03:30
03:40
03:5

Head to the log files to observe the conversations between agents, their movements across the maze, the prompts sent to the LLMs, and how the LLMs are managing the scheduling.

That concludes this tutorial series. While we've covered the basics to make the original Simulacra code more accessible, there's much more to explore. For those interested in deepening their understanding, I encourage you to delve into the original codebase and accompanying paper.